# LGBM Model

The winning solution uses combination of four models and lgbm is one of the model. I will start by looking at this solution. 

[Winning Solution](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/47582#latest-605356)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""

from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

import sys

sys.path.append('..')

from retail_sales_prediction.utils.data_loader import readDataStore

from retail_sales_prediction.utils.data_preparation import FeaturePreparation

from retail_sales_prediction.logging_configurator import LoggingConfigurator

from retail_sales_prediction.utils.run_model import run_model_lgbm

from retail_sales_prediction import logger

In [4]:
data_dir = '/media/farmshare2/Research/raza/p_data/'

LoggingConfigurator.init(data_dir + 'logs.log')
logger.info('Logger started')

logger.info('Loading the Read Data Module')

rd = readDataStore(data_dir)
# Reading the test data
df_test = rd.read_test('test.csv')
logger.info('Test Data Loaded')

# Read the items data frame
df_items = rd.read_items('items.csv').set_index("item_nbr")
logger.info('Items Data Loaded')

# Read the stores data frame
df_stores = rd.read_items('stores.csv').set_index("store_nbr")
logger.info('Stores Data Loaded')

# Reading the training data
df_train = rd.read_train('train.csv')
logger.info('Training Data Loaded')


2019-09-01 15:41:32 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:4 - Logger started
2019-09-01 15:41:32 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:6 - Loading the Read Data Module
2019-09-01 15:41:36 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:11 - Test Data Loaded
2019-09-01 15:41:36 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:15 - Items Data Loaded
2019-09-01 15:41:36 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:19 - Stores Data Loaded
2019-09-01 15:46:13 INFO  [retail_sales_prediction] <ipython-input-4-a3dbcb498f24>:23 - Training Data Loaded


In [5]:
feature_prep = FeaturePreparation(df_train, df_test, df_items, df_stores)

2019-09-01 15:48:49 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:22 - Pre Processing the data for training and promotions
Converting Label to values
2019-09-01 15:48:49 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:35 - Performing the label to values conversion
2019-09-01 15:49:14 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:56 - promo_2017_train
2019-09-01 15:49:16 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:74 - Merging the promotion data for whole 2017 year 
2019-09-01 15:49:34 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:94 - Working on Unit_Sales data
2019-09-01 15:49:56 INFO  [retail_sales_prediction.utils.data_preparation] data_preparation.py:121 - Data Preparation step Completed


In [8]:
X_train, y_train = feature_prep.get_training_data(anchor_date=date(2017, 6, 14), num_days=6)

Preparing dataset...


KeyError: "None of [DatetimeIndex(['2017-05-31', '2017-06-01', '2017-06-02', '2017-06-03',\n               '2017-06-04', '2017-06-05', '2017-06-06', '2017-06-07',\n               '2017-06-08', '2017-06-09', '2017-06-10', '2017-06-11',\n               '2017-06-12', '2017-06-13'],\n              dtype='datetime64[ns]', freq='D')] are in the [columns]"

In [ ]:


X_train, y_train = feature_prep.get_training_data(anchor_date=date(2017, 6, 14), num_days=6)

X_val, y_val = feature_prep.get_validation_data(val_start_date=date(2017, 7, 26))

X_test = feature_prep.get_test_data(test_start_date=date(2017, 7, 26))

run_model_lgbm(feature_prep, X_train, y_train,X_val, y_val, X_test, num_days=6)


In [ ]:
feature_prep = FeaturePreparation(df_train, df_test, df_items, df_stores)

In [ ]:
print("Training and predicting models...")
params = {
    'num_leaves': 3,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 20
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

y_val = np.array(val_pred).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index,
    columns=pd.date_range("2017-07-26", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
df_preds["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
df_preds.reset_index().to_csv('lgb_cv.csv', index=False)





print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)


submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb_sub.csv', float_format='%.4f', index=None)

In [ ]:
y_val[:,i].shape

In [ ]:
print("Validation mse:", mean_squared_error(
    y_val[:,i], np.array(val_pred).transpose()))